In [2]:
import osmium
import json

class OSMHandler(osmium.SimpleHandler):
    def __init__(self):
        super(OSMHandler, self).__init__()
        self.features = []

    def node(self, n):
        # 篩選需要的node屬性
        properties = {
            'id': n.id,
            'lat': n.location.lat,
            'lon': n.location.lon,
            'tags': dict(n.tags),
        }

        # 建立GeoJSON Feature
        feature = {
            'type': 'Feature',
            'geometry': {
                'type': 'Point',
                'coordinates': [n.location.lon, n.location.lat]
            },
            'properties': properties
        }

        self.features.append(feature)

    def way(self, w):
        # 篩選需要的way屬性
        properties = {
            'id': w.id,
            'tags': dict(w.tags),
        }

        # 建立GeoJSON Feature
        feature = {
            'type': 'Feature',
            'geometry': {
                'type': 'LineString',
                'coordinates': [[n.location.lon, n.location.lat] for n in w.nodes]
            },
            'properties': properties
        }

        self.features.append(feature)

    def relation(self, r):
        # 篩選需要的relation屬性
        properties = {
            'id': r.id,
            'tags': dict(r.tags),
        }

        # 建立GeoJSON Feature
        feature = {
            'type': 'Feature',
            'geometry': {
                'type': 'Polygon',
                'coordinates': [[n.location.lon, n.location.lat] for n in r.members if n.role == 'outer']
            },
            'properties': properties
        }

        self.features.append(feature)

# 讀取OSM檔案
input_file = 'pu.osm'
handler = OSMHandler()
handler.apply_file(input_file)

# 建立GeoJSON FeatureCollection
feature_collection = {
    'type': 'FeatureCollection',
    'features': handler.features
}

# 寫入JSON檔案
output_file = 'output.json'
with open(output_file, 'w') as f:
    json.dump(feature_collection, f)


InvalidLocationError: invalid location